# Exception handling
 
So far we've seen that Python "throws" errors when something isn't to its liking. For example, when we divide by zero or simply mistype something. These errors can (mostly) be "caught", meaning they can be handled, so instead of the program stopping we can do something else.
 
This feature is such an integral part of Python that often this is the recommended approach rather than pre-checking whether a code fragment will run with the given data.

In [ ]:
# read a number and print its reciprocal
data = input("Enter a number: ")
reciprocal = 1 / float(data)
print("reciprocal:", reciprocal)

The above code works perfectly well as long as the user is cooperative and knowledgeable. However, as soon as they type 0, the code will crash with an error. If you try it, you'll also see the name of the error: ZeroDivisionError.
 
Of course we could do something like put a check before calculating the reciprocal:
```python
if data == 0:
  print("cannot compute reciprocal!")
elif:
  # previous code

```
But Python's suggestion is rather to let the error occur and then handle it. We can do this with a try/except structure:
 


In [ ]:
data = input("Enter a number: ")
try:
  reciprocal = 1 / float(data)
except ZeroDivisionError:
  print("Cannot divide by zero!")
else:
  print("reciprocal:", reciprocal)

mondj egy számot: 21
reciprok: 0.047619047619047616


What we put inside the `try:` block is attempted, and Python checks whether we handle that specific error type, i.e. whether there is a corresponding `except` part. If it finds such an except it runs it. If it doesn't find one, it still raises the error as before. For example, if the user types `meggybefőtt` it won't be able to convert that to a float and we'll still get an error (try it to see what kind!).
 
As you can see, our try structure can also have an `else:` branch which runs if no error occurred. (Of course we could have just put the code after the try block, but sometimes we don't want to catch errors from other parts.)
 
We can place any number of except branches in the try structure so we can handle all the error types that we think might reasonably occur:

In [ ]:
data = input("Enter a number: ")
try:
  reciprocal = 1 / float(data)
except ZeroDivisionError:
  print("Cannot divide by zero!")
except ValueError:
  print("You didn't enter a number!")
else:
  print("reciprocal:", reciprocal)

Be careful though: Python checks the `except` blocks in order, so if you want to examine a broader error and a narrower error, put the narrower one first, otherwise the broader one will catch it (and the narrower one will never run). So this will not work (you can't get the specific error message even if you typed text instead of a number):

In [ ]:
try:
  int(input("Please enter a number!")) # if you don't provide a number, ValueError occurs
except Exception:  # we catch every error
  print("Oh, something went wrong!")
except ValueError: # we never get here because the above Except caught it!
  print("You didn't enter a number!")


Fix the above code and bring out the specific error message (for example by entering "negyvenkettő")!

You can even raise an error yourself if you want! You can use built-in exceptions or create your own. In the except part you can give a name to the exception object, so you can print its message or do something else with it.

In [ ]:
import math
 
try:
  pos_num = float(input("Enter a positive number: "))
  if pos_num <= 0: # if the number is not positive we raise an error ourselves!
    raise ValueError("The provided number is not positive!")
  print("Logarithm of the number:", math.log(pos_num))
 
except ValueError as e:
  print(e)


You can catch errors anywhere, even mistakes in the program structure itself:

In [ ]:
try:
  import no_such_lib
except ModuleNotFoundError as e:
  print("Warning!", e)
print("the program continues normally")


Figyelem! No module named 'nincsilyenlib'
nyugodtan megy tovább a program


In [ ]:
try:
  print(no_such_variable)
except NameError as e:
  print("Warning!", e)
print("the program continues normally")


Figyelem! name 'nincsilyenvaltozo' is not defined
nyugodtan megy tovább a program


In [ ]:
try:
  a,b,c = 1,2
except ValueError as e:
  print("not enough values!")
print("But the program continues...")


nincs elég sok érték!
De a program megy tovább...


Errors have a whole hierarchy, you could say a family tree. You don't have to handle every error individually; sometimes it's enough to handle the broader category. File handling is a classic example where many different kinds of errors can occur. You'd think writing to or reading from a file is simple, but many problems can happen! Is the disk full? Is the directory missing? Does the filename not exist? Do we lack permission to write? Does the file already exist? Etc.


In [ ]:
# read data from a file
f = open('adat.txt') # open the file
for line in f: # for each line ...
  print(line, end='') # print the lines
f.close() # close the file.

kaktusz
ciklámen
nárcisz


The above code ends sadly because unfortunately no file with that name exists.
Fix the code! Wrap it in a try/except block that handles the printed error and prints a nice Hungarian message! (you can see the exact error name in the error message).

In [ ]:
# create the file so the previous code can read from it
f = open('adat.txt', "w")
f.write("cactus\n")
f.write("cyclamen\n")
f.write("daffodil\n")
f.close()

In [ ]:
!rm adat.txt

Now you can go back and run the previous code block.
 
Of course it might not be exactly that error; for example you might not have permission (PermissionError), the path might be a directory (IsADirectoryError), or the disk might have fried (well, that's unlikely on Colab). These are all OSError (operating system error), so if you just catch OSError you've handled all such possible problems!
 
Actually every existing error inherits from BaseException, so if you catch that you catch everything.

In [ ]:
try:
  1 / 0.0  # this is also caught
  math.log(-100) # and this too
  open('nincsilyen.txt') # or this
  unknown_code() # or this
  a = []
  a[20] # even this (IndexError)
except BaseException as e:
  print("Warning!", e)
print("we move on...")

Figyelem! list index out of range
megyünk tovább...


You may feel a strong urge to wrap the entire program in one huge try: block and catch every error with an except. Even if that would "work", don't do it. Python errors are our good friends because they indicate precisely that something happened which we did not plan for. When an error occurs (and the program stops) it also gives us a guide (the so-called stack trace) that tells exactly where in our program the problem happened.
 
If we prematurely catch and "swallow" errors we will never know that something went wrong and our program will still likely not behave correctly!

## Exception propagation
 
If an exception happens inside a called function and it is not handled there, the function's execution is interrupted and the exception is passed to the caller (the place where the function was invoked). If that caller doesn't handle it either, it propagates to its caller, and so on until we reach the main program. If the main program doesn't handle it, the Python script terminates and prints the exception. The exception also remembers where it originated, so you can trace the entire call tree. This is extremely useful for finding where the problem is.

In [ ]:
import traceback
def f(): # this extremely useful function always raises an exception.
  raise Exception("This is an exception!")
 
def g():
  # we do some other work...
  f()
  # and here too...
 
def h():
  g()
  # other code...
 
try:
  h() # which calls g which calls f.
except:
  # print the caught exception
  print(traceback.format_exc())


If you run the above code and carefully read the exception contents, you'll learn a lot. For example, Colab executed this code by creating a temporary file (you can see the file name). Then you'll see the function call order (stack trace), letting you trace back where the error occurred.
 
At the bottom of the list you'll find the actual error, and above it the sequence of functions that called the error-causing function without handling that specific exception.

## Context managers
 
In the small file handling example we saw that programs need to close files. If a file remains open it's not good: you can't have an unlimited number of open files (so eventually you'll run into trouble), and data written to an open file might still be in memory (in a buffer) and not yet on disk, so when you exit it could be lost. Also every open file consumes memory, so leaving them open is bad.
 
But there's a problem! What if an error occurs after you've opened the file? Because of the error (even if you catch it) the interpreter will skip a large part of your code, and it might skip the close too!
 
```python
try:
  f = open('adatok.txt')
  # many reads and calculations
  # which might raise an error here, and execution jumps to except!
  # we continue computing
  f.close() # and we would close, but this gets skipped...
except SevereError
  # we handle the error, but the file remained open...
  # so we would need to close it here as well
except AnotherError
  # and here too, etc. etc.
```
 
To avoid having to constantly check whether an error occurred before the close, context managers were introduced. The `with` keyword opens a context for us that automatically performs cleanup (in our case closes the file) whether the block completed successfully or not!

In [ ]:
# This program sums the numbers in a file
try:
  # with opens a context that automatically closes the file
  with open('adat.txt') as f: # like f=open('adat.txt') but with cleanup
    total = 0
    for line in f: # for every line
      total = total + float(line) # add to the total
    print("Total:", total)
 
except: # we catch everything! (You shouldn't do this though :)
  print('Is it really closed?', f.closed)

If you run this code block it won't print the sum because the file we created earlier contains plant names that cannot be converted to float (and thus summed), so it fails in the middle with a ValueError which we nicely catch. However, because we cleverly opened the file with a with block, it was closed automatically!
 
Note that we don't try to close it at the end — no need: the context manager always closes it, whether an error occurred or everything ran fine!